In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
### Load label
labels = pd.read_csv("../data/label.csv")

### load data
data = pd.read_csv("../data/sensor_log.csv")

In [3]:
labels.head()

,block_id,anomalous
0,144,0
1,145,0
2,146,0
3,147,0
4,148,0


In [4]:
data.head()

,timestamp,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2018-04-01 00:00:00,0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353
1,2018-04-01 00:01:00,0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353
2,2018-04-01 00:02:00,0,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013
3,2018-04-01 00:03:00,0,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247
4,2018-04-01 00:04:00,0,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247


In [5]:
data.describe()

,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
count,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000
mean,9641.000000,2.459135,48.153622,51.790473,44.226571,633.010905,75.725928,13.943126,16.195575,15.489414,15.101593
std,5566.537047,0.057049,2.144784,1.591106,1.549972,11.372609,7.241880,0.662427,0.370934,0.361631,0.225652
min,0.000000,0.000000,37.065970,42.317710,34.982640,3.104745,0.000000,0.014468,0.028935,0.028935,0.007234
25%,4820.000000,2.448669,46.701390,50.781250,43.185760,628.935200,70.811155,13.411460,15.957750,15.227140,15.053530
50%,9641.000000,2.456539,48.263885,51.866320,44.357635,633.680500,75.749880,13.686340,16.167530,15.494790,15.082470
75%,14462.000000,2.500810,49.522570,52.951385,45.355900,638.194500,80.478037,14.576100,16.427950,15.697340,15.118630
max,19282.000000,2.585285,60.237007,56.032990,50.204142,690.462452,113.435420,15.964990,17.502495,18.695345,16.399020


In [6]:
id = labels[labels['anomalous'] == 1]['block_id'].values

In [7]:
block_id = np.unique(data[data['sensor_06'] < 10]['block_id'].values)

In [8]:
len(block_id)

5

In [9]:
len(set(block_id) - set(id))

4

In [10]:
len(set(block_id) - set(labels['block_id'].values))

4

In [11]:
blockid_train = labels['block_id'].values

In [12]:
_train = data['block_id'].isin(blockid_train)
data_train = data[_train]
data_test = data[~_train]

In [13]:
anomalous = labels['anomalous'].values

In [14]:
unique, count = np.unique(anomalous, return_counts=True)
print(unique)
print(count)

[0 1]
[5617  164]


In [15]:
data_train.to_csv('../data/train.csv', index=False)
data_test.to_csv('../data/test.csv', index=False)

### Scale Min-Max

In [56]:
scaler = MinMaxScaler()

In [57]:
columns = list(data.columns)

In [58]:
train_values = data_train.values[:, 2:]
test_values = data_test.values[:, 2:]

block_train_values = np.expand_dims(data_train.values[:, 1], axis=1)
block_test_values = np.expand_dims(data_test.values[:, 1], axis=1)

timestemp_train_values = np.expand_dims(data_train.values[:, 0], axis=1)
timestemp_test_values = np.expand_dims(data_test.values[:, 0], axis=1)

In [59]:
train_values_scaled = scaler.fit_transform(train_values)
test_values_scaled = scaler.fit_transform(test_values)

In [60]:
timestemp_train_values.shape, block_train_values.shape, train_values_scaled.shape

((57810, 1), (57810, 1), (57810, 10))

In [61]:
train_values_all = np.hstack((timestemp_train_values, block_train_values, train_values_scaled))
test_values_all = np.hstack((timestemp_test_values, block_test_values, test_values))

In [62]:
df_train = pd.DataFrame(data=train_values_all, columns=columns)
df_test = pd.DataFrame(data=test_values_all, columns=columns)

In [63]:
df_train.to_csv('../data/train_normalize.csv', index=False)
df_test.to_csv('../data/test_normalize.csv', index=False)

In [64]:
len(df_train), len(df_test)

(57810, 135020)

In [65]:
df = pd.read_csv('../data/train_normalize.csv')
df.head()

,timestamp,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2018-04-02 00:00:00,144,0.951342,0.341364,0.755172,0.741935,0.896145,0.791095,0.778336,0.581489,0.263138,0.488599
1,2018-04-02 00:01:00,144,0.951342,0.341364,0.755172,0.741935,0.896145,0.791095,0.778336,0.581489,0.263138,0.488599
2,2018-04-02 00:02:00,144,0.947537,0.337395,0.751724,0.741935,0.886458,0.796020,0.780570,0.570086,0.279793,0.488599
3,2018-04-02 00:03:00,144,0.953245,0.341364,0.751724,0.741935,0.881025,0.806212,0.775544,0.570086,0.263138,0.472316
4,2018-04-02 00:04:00,144,0.945634,0.331441,0.755172,0.741935,0.893546,0.782298,0.782245,0.570086,0.336418,0.488599


### Mean data train

In [101]:
data_train_mean = df_train.groupby("block_id",sort=False).mean()

C:\Users\Admin\AppData\Local\Temp\ipykernel_2328\2815326811.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_train_mean = df_train.groupby("block_id",sort=False).mean()


In [102]:
data_train_mean

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
block_id,,,,,,,,,,
144,0.949516,0.335013,0.753448,0.744803,0.885915,0.798645,0.778392,0.578524,0.295115,0.482411
145,0.949439,0.340570,0.754138,0.750896,0.880741,0.822409,0.773255,0.578981,0.294949,0.470685
146,0.948983,0.338387,0.754138,0.754839,0.885277,0.801598,0.776047,0.578297,0.308772,0.476222
147,0.949554,0.335609,0.754138,0.758423,0.884796,0.776834,0.781798,0.582858,0.309937,0.471989
148,0.948716,0.341364,0.754483,0.764158,0.886364,0.784078,0.780402,0.581033,0.301277,0.474268
...,...,...,...,...,...,...,...,...,...,...
18705,0.955909,0.449728,0.494828,0.593548,0.879079,0.843773,0.895533,0.736327,0.296780,0.479807
18706,0.956061,0.450522,0.490345,0.594982,0.879024,0.796388,0.896482,0.740888,0.287454,0.485017
18707,0.955909,0.452308,0.489310,0.594982,0.874780,0.774932,0.898883,0.733362,0.314101,0.476223


In [103]:
labels

,block_id,anomalous
0,144,0
1,145,0
2,146,0
3,147,0
4,148,0
...,...,...
5776,18705,0
5777,18706,0
5778,18707,0
5779,18708,0


### Split train val

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
block_id = labels['block_id'].values

In [20]:
x_train, x_val, y_train, y_val = train_test_split(block_id, labels, test_size=0.2)

In [24]:
x_train.shape, x_val.shape

((4624,), (1157,))

In [25]:
y_val.groupby('anomalous').count()

,block_id
anomalous,
0,1126
1,31


In [26]:
y_train.groupby('anomalous').count()

,block_id
anomalous,
0,4491
1,133


In [55]:
path = 'E:/prdcv/competition/src/LSTM_Trans/'

In [56]:
df_train = data_train[data_train['block_id'].isin(x_train)]
df_val = data_train[data_train['block_id'].isin(x_val)]

In [57]:
len(df_train), len(df_val)

(46240, 11570)

In [58]:
df_train.drop(columns=['timestamp'], inplace=True)
df_val.drop(columns=['timestamp'], inplace=True)

df_train.to_csv(path + 'train.csv', index=False)
y_train.to_csv(path + 'train_labels.csv', index=False)

df_val.to_csv(path + 'val.csv', index=False)
y_val.to_csv(path + 'val_labels.csv', index=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10576\1240652348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=['timestamp'], inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10576\1240652348.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val.drop(columns=['timestamp'], inplace=True)


In [59]:
df_train

,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
1450,145,2.455556,45.572910,52.34375,44.878471,633.101900,83.14183,13.14381,16.13136,15.69734,15.05353
1451,145,2.449653,45.833330,52.34375,44.878471,633.217600,80.72942,13.01360,16.16753,15.56713,15.08247
1452,145,2.464410,45.659720,52.34375,44.878471,648.842500,82.60482,12.99913,16.21094,15.65393,15.11863
1453,145,2.444734,46.006940,52.34375,44.878471,620.717500,81.10479,13.05700,16.13136,15.61777,15.08970
1454,145,2.466377,45.659720,52.30035,44.878470,627.430500,83.35425,13.02807,16.12413,15.81308,15.01013
...,...,...,...,...,...,...,...,...,...,...,...
187095,18709,2.475232,48.350693,49.04514,42.968750,624.421265,72.64686,14.67737,16.65220,15.53096,15.05353
187096,18709,2.468345,48.350693,49.00174,42.968750,627.314800,73.31248,14.64844,16.65220,15.56713,15.08247
187097,18709,2.471296,48.350693,49.04514,42.968750,631.365700,74.63629,14.60503,16.60880,15.56713,15.05353
187098,18709,2.471296,48.350693,49.04514,42.968750,631.365700,76.36526,14.60503,16.60880,15.81308,15.08247


In [68]:
tmp = df_train

In [73]:
from sklearn.preprocessing import RobustScaler

In [74]:
RS = RobustScaler()
train = RS.fit_transform(df_train)

In [76]:
train.shape

(46240, 11)

In [79]:
labels

,block_id,anomalous
0,144,0
1,145,0
2,146,0
3,147,0
4,148,0
...,...,...
5776,18705,0
5777,18706,0
5778,18707,0
5779,18708,0


In [80]:
lb = labels['anomalous'].to_numpy()

In [82]:
lb = lb[:, None]

In [83]:
lb.shape

(5781, 1)

In [84]:
np.repeat(lb, 10, axis=1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [89]:
num_feature = list(df_train.columns)[1:]

In [90]:
num_feature

['sensor_00',
 'sensor_01',
 'sensor_02',
 'sensor_03',
 'sensor_04',
 'sensor_05',
 'sensor_06',
 'sensor_07',
 'sensor_08',
 'sensor_09']

In [92]:
tmp = tmp.groupby('block_id')[num_feature].diff().add_prefix('diff_')

In [95]:
tmp = tmp.fillna(0)

In [99]:
value = tmp.iloc[:10].mean(axis=0).values

In [100]:
tmp.iloc[0] = value

In [101]:
tmp

,diff_sensor_00,diff_sensor_01,diff_sensor_02,diff_sensor_03,diff_sensor_04,diff_sensor_05,diff_sensor_06,diff_sensor_07,diff_sensor_08,diff_sensor_09
1450,0.000492,0.043403,0.0000,0.004340,-0.289350,-0.30077,-0.014468,0.007958,0.00000,0.00000
1451,-0.005903,0.260420,0.0000,0.000000,0.115700,-2.41241,-0.130210,0.036170,-0.13021,0.02894
1452,0.014757,-0.173610,0.0000,0.000000,15.624900,1.87540,-0.014470,0.043410,0.08680,0.03616
1453,-0.019676,0.347220,0.0000,0.000000,-28.125000,-1.50003,0.057870,-0.079580,-0.03616,-0.02893
1454,0.021643,-0.347220,-0.0434,-0.000001,6.713000,2.24946,-0.028930,-0.007230,0.19531,-0.07957
...,...,...,...,...,...,...,...,...,...,...
187095,0.008855,0.000000,0.0434,0.000000,0.000000,3.25138,0.028930,-0.101270,-0.08681,0.00000
187096,-0.006887,0.000000,-0.0434,0.000000,2.893535,0.66562,-0.028930,0.000000,0.03617,0.02894
187097,0.002951,0.000000,0.0434,0.000000,4.050900,1.32381,-0.043410,-0.043400,0.00000,-0.02894
187098,0.000000,0.000000,0.0000,0.000000,0.000000,1.72897,0.000000,0.000000,0.24595,0.02894


In [103]:
df_train = pd.concat([df_train, tmp], axis=1)

In [115]:
df_train.to_csv('./LSTM_Trans/data/train.csv', index=False)

In [116]:
df_train

,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,diff_sensor_00,diff_sensor_01,diff_sensor_02,diff_sensor_03,diff_sensor_04,diff_sensor_05,diff_sensor_06,diff_sensor_07,diff_sensor_08,diff_sensor_09
1450,145,2.455556,45.572910,52.34375,44.878471,633.101900,83.14183,13.14381,16.13136,15.69734,...,0.000492,0.043403,0.0000,0.004340,-0.289350,-0.30077,-0.014468,0.007958,0.00000,0.00000
1451,145,2.449653,45.833330,52.34375,44.878471,633.217600,80.72942,13.01360,16.16753,15.56713,...,-0.005903,0.260420,0.0000,0.000000,0.115700,-2.41241,-0.130210,0.036170,-0.13021,0.02894
1452,145,2.464410,45.659720,52.34375,44.878471,648.842500,82.60482,12.99913,16.21094,15.65393,...,0.014757,-0.173610,0.0000,0.000000,15.624900,1.87540,-0.014470,0.043410,0.08680,0.03616
1453,145,2.444734,46.006940,52.34375,44.878471,620.717500,81.10479,13.05700,16.13136,15.61777,...,-0.019676,0.347220,0.0000,0.000000,-28.125000,-1.50003,0.057870,-0.079580,-0.03616,-0.02893
1454,145,2.466377,45.659720,52.30035,44.878470,627.430500,83.35425,13.02807,16.12413,15.81308,...,0.021643,-0.347220,-0.0434,-0.000001,6.713000,2.24946,-0.028930,-0.007230,0.19531,-0.07957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187095,18709,2.475232,48.350693,49.04514,42.968750,624.421265,72.64686,14.67737,16.65220,15.53096,...,0.008855,0.000000,0.0434,0.000000,0.000000,3.25138,0.028930,-0.101270,-0.08681,0.00000
187096,18709,2.468345,48.350693,49.00174,42.968750,627.314800,73.31248,14.64844,16.65220,15.56713,...,-0.006887,0.000000,-0.0434,0.000000,2.893535,0.66562,-0.028930,0.000000,0.03617,0.02894
187097,18709,2.471296,48.350693,49.04514,42.968750,631.365700,74.63629,14.60503,16.60880,15.56713,...,0.002951,0.000000,0.0434,0.000000,4.050900,1.32381,-0.043410,-0.043400,0.00000,-0.02894
187098,18709,2.471296,48.350693,49.04514,42.968750,631.365700,76.36526,14.60503,16.60880,15.81308,...,0.000000,0.000000,0.0000,0.000000,0.000000,1.72897,0.000000,0.000000,0.24595,0.02894


In [107]:
tmp = df_val
tmp = tmp.groupby('block_id')[num_feature].diff().add_prefix('diff_')

In [108]:
tmp = tmp.fillna(0)

In [109]:
value = tmp.iloc[:10].mean(axis=0).values

In [110]:
tmp.iloc[0] = value

In [111]:
tmp

,diff_sensor_00,diff_sensor_01,diff_sensor_02,diff_sensor_03,diff_sensor_04,diff_sensor_05,diff_sensor_06,diff_sensor_07,diff_sensor_08,diff_sensor_09
1440,-0.000393,-0.026042,0.00000,8.680725e-03,-0.648140,0.456602,0.005064,-0.003617,0.020255,-0.00651
1441,0.000000,0.000000,0.00000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1442,-0.009838,-0.086810,-0.04340,0.000000e+00,-4.745300,0.489120,0.028940,-0.036170,0.072340,0.00000
1443,0.014757,0.086810,0.00000,0.000000e+00,-2.662100,1.012390,-0.065110,0.000000,-0.072340,-0.03616
1444,-0.019676,-0.217010,0.04340,0.000000e+00,6.134300,-2.375320,0.086810,0.000000,0.318290,0.03616
...,...,...,...,...,...,...,...,...,...,...
187065,0.002951,0.000000,0.04340,0.000000e+00,-0.463000,0.579650,0.036170,0.000000,-0.166380,-0.07234
187066,-0.002951,0.000000,-0.04340,0.000000e+00,1.852000,1.310490,0.072340,0.050640,0.000000,-0.00723
187067,0.006887,0.000000,0.04340,0.000000e+00,8.680500,0.637140,0.000000,0.000000,0.202550,0.07957
187068,-0.006887,0.000000,-0.04340,0.000000e+00,-14.855350,-1.275080,0.028930,-0.050640,-0.202550,-0.04341


In [112]:
df_val = pd.concat([df_val, tmp], axis=1)

In [114]:
df_val.to_csv('./LSTM_Trans/data/val.csv', index=False)

In [117]:
len(df_val)

11570